# Wrangle Act
### By Julio Uribe

The purpose of this project is to exapnd on wrangling abilities. In this file we will gather data about "weRateDogs" twitter posts from a couple of different sources: directly from Twitter API, using the provided twitter enhanced file for tweet id's, and pulling from udacity's server to look at neural net results in a tsv file.

# Setup: Import  Modules

In [1]:
import tweepy
from tweepy import OAuthHandler
import json
import numpy as np
from timeit import default_timer as timer
import pandas as pd
import requests

# Gather Data

## First Source File: Twitter Enhanced file and set up API keys

In [2]:
#load file info into dataframe for tweet id's to use later
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
#load tweet ids for api extraction
tweet_ids = twitter_archive.tweet_id.values

#Set up API credentials from file outside directory
creds = []
with open('/Users/Jules/Desktop/DAND/twitter_credentials.txt', 'r') as f:
    creds = f.read().split("'")
consumer_key = creds[1]
consumer_secret = creds[3]
access_token = creds[5]
access_secret = creds[7]
#create auth object with keys
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
#create tweepy api object for requests
api = tweepy.API(auth, wait_on_rate_limit = True)

print (len(tweet_ids))

2356


## Second Source File: Query Twitter's API for JSON data

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # Loop pauses/resumes at about 900 tweets due to api's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

### Load JSON data we got from Twitter API into a cleaner dataframe

In [3]:
#load twitter json file into a pandas dataframe
tweets_json_full = pd.read_json("tweet_json.txt", lines=True)
#tweets_json_full.info()

In [4]:
#create a smaller version of tweets_json_full with only the columns we're interested in
tweets_json = pd.DataFrame(tweets_json_full[['id', 'created_at', 'favorite_count', 'retweet_count', 'full_text', 'extended_entities']])
#tweets_json.head()

In [5]:
#seeing if we can extract anything interesting from the extended_entities values
# for i in range(5):
#     print(tweets_json_full['extended_entities'][i]['media'][0]['url'])

## Third Source file: Use Requests Module to Load Neural Net Results

In [6]:
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
with open("image_predictions.tsv", 'wb') as file:
    for chunk in r.iter_content(chunk_size=128):
        file.write(chunk)
#read file back in and create a df for image predictions data     
image_predictions = pd.read_csv("image_predictions.tsv", sep='\t')

# Assessing Data

For this project, we have three dataframes we're currently working with: 
* twitter_archive - imported tweet info from twitter_archive_enhanced.txt provided by udacity. Has tweet IDs, tweet text, ratings, and other information.
* tweets_json - data from twitter API containing retweets, favorited count, tweet text, and more.
* image_predictions - results from neural net. Contains three predictions, image url, number of images, etc.

The three dataframes provide info about the tweets posted from the WeRateDogs twitter profile. We'll do some assessing of the data before we merge these dataframes together. Then we'll clean the data to get the most complete data set we can.

### Tidiness Issues
* twitter_archive has 2356 entries, tweets_json has 2340 entries, and image_predictions has 2075 entries. They all serve the same purpose of giving us more info about individual tweets. Should merge all three
* In 'twitter_archive', the last four columns (doggo, floofer, pupper, puppo) are not always observed and best serve as a category. We should combine these 4 columns into one

### Quality Issues
##### *twitter_archive*
* Doggo, pupper, floofer, puppo cols will become redundant once we add new 'stage' column
* 'None' values are strings but should be NaN values. True for all three tables
* Incorrect and missing names under 'name' column: 'None', 'a', 'the', 'an', etc.
* Rating numerators are under 10 but according to the twitter profile, all ratings should be above 10
* Ther are rows where the 'rating_denominator' is lower or higher than 10. We need to standardize all rows to be out of 10.
* Timestamp column is in string format instead of datetime.

##### *tweets_json*
* 'Id' column should be renamed to 'tweet_id' to be consistent with other two dataframes
* duplicate 'created_at' column that is equivalent to twitter_archive 'timestamp' column. We'll keep one
* we'll have duplicate 'full_text' columns as well after merging

##### *image_predictions*
* Dog breed predictions from the neural net in p1 give us results in inconsistent lower/upper case usage
* We get results from our neural net in p1 that don't include dogs such as 'desktop_computer', 'electric_fan', 'wild_boar'.
* Some prediction of dog breeds aren't actual dog breeds

In [7]:
#twitter_archive

In [8]:
# twitter_archive has 2356 entries, tweets_json has 2340 entries, and image_predictions has 2075 entries. We'll merge later
# on tweet IDs.
#twitter_archive.describe()
#tweets_json.info()
#twitter_archive.isnull().sum()
#type(twitter_archive.timestamp[0])
#twitter_archive.source.value_counts()

In [9]:
#twitter_archive.name.value_counts()

In [10]:
#twitter_archive.rating_denominator.value_counts()

In [11]:
#twitter_archive.rating_numerator.value_counts()

In [12]:
tweets_json

,id,created_at,favorite_count,retweet_count,full_text,extended_entities
0,892420643555336193,2017-08-01 16:23:56,37806,8244,This is Phineas. He's a mystical boy. Only eve...,"{'media': [{'id': 892420639486877696, 'id_str'..."
1,892177421306343426,2017-08-01 00:17:27,32488,6091,This is Tilly. She's just checking pup on you....,"{'media': [{'id': 892177413194625024, 'id_str'..."
2,891815181378084864,2017-07-31 00:18:03,24452,4029,This is Archie. He is a rare Norwegian Pouncin...,"{'media': [{'id': 891815175371796480, 'id_str'..."
3,891689557279858688,2017-07-30 15:58:51,41150,8389,This is Darla. She commenced a snooze mid meal...,"{'media': [{'id': 891689552724799489, 'id_str'..."
4,891327558926688256,2017-07-29 16:00:24,39341,9094,This is Franklin. He would like you to stop ca...,"{'media': [{'id': 891327551943041024, 'id_str'..."
5,891087950875897856,2017-07-29 00:08:17,19777,3015,Here we have a majestic great white breaching ...,"{'media': [{'id': 891087942176911360, 'id_str'..."
6,890971913173991426,2017-07-28 16:27:12,11562,1993,Meet Jax. He enjoys ice cream so much he gets ...,"{'media': [{'id': 890971906207338496, 'id_str'..."
7,890729181411237888,2017-07-28 00:22:40,63762,18315,When you watch your owner call another dog a g...,"{'media': [{'id': 890729118844600320, 'id_str'..."
8,890609185150312448,2017-07-27 16:25:51,27182,4146,This is Zoey. She doesn't want to be one of th...,"{'media': [{'id': 890609177319665665, 'id_str'..."
9,890240255349198849,2017-07-26 15:59:51,31161,7159,This is Cassie. She is a college pup. Studying...,"{'media': [{'id': 890240245463175168, 'id_str'..."


In [13]:
# tweets_json.isnull().sum()
# tweets_json.info()

In [14]:
# image_predictions.isnull().sum()
# image_predictions

In [15]:
# image_predictions.describe()

In [16]:
#image_predictions.p1.value_counts()
# image_predictions.p1.value_counts()
#image_predictions.p3.value_counts()

In [17]:
# if it hits false multiple times, toss out row
#image_predictions[image_predictions.p3 == 'space_shuttle']

In [18]:
#image_predictions[image_predictions.p1 == 'coffee_mug'].jpg_url

In [19]:
# there's a good chance that a large part of our data set doesn't actually contain dogs in the image, throwing off ratings
#image_predictions['p1_dog'].mean(), image_predictions['p2_dog'].mean(), image_predictions['p3_dog'].mean()

In [20]:
# explore prediction results for tweets with more than one image. How does the neural net handle multiple images?
# multi_pic = image_predictions[image_predictions["img_num"] > 1]
# multi_pic

In [21]:
# lets compare the average p1_dog, p2_dog, p3_dog rates from multiple images to the whole dataframe
# multi_pic['p1_dog'].mean(), multi_pic['p2_dog'].mean(), multi_pic['p3_dog'].mean()
# Multiple images is more likely to have a dog in it than the general dog prediciton rate from entire dataframe

In [22]:
#checking for duplicated values
# twitter_archive[twitter_archive.tweet_id.duplicated()]
# tweets_json[tweets_json.id.duplicated()]
# image_predictions[image_predictions.tweet_id.duplicated()]

# Cleaning Data

### Create copies for data

In [23]:
# Create copies of all three dataframes
twitter_archive_clean = twitter_archive.copy()
tweets_json_clean = tweets_json.copy()
image_predictions_clean = image_predictions.copy()

### First Merge and Cleaning

In [24]:
# Define
# Rename 'id', 'created_at', and 'text' in the dataframes for consistency
# Clean
tweets_json_clean.rename(columns={'id':'tweet_id', 'created_at': 'timestamp'}, inplace=True)
twitter_archive_clean.rename(columns={'text':'full_text', 'created_at': 'timestamp'}, inplace=True)
# Test: Make sure column names are consistent when shared/overlapping
#twitter_archive_clean.columns, tweets_json_clean.columns

In [25]:
# Define
# Drop extra columns that are less interesting and only keep data that is mostly complete and easy to interpret
#Clean
twitter_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'], axis=1, inplace=True)
tweets_json_clean.drop(['extended_entities'], axis=1, inplace=True)
# Test: Make sure all the appropriate columns have been deleted
#twitter_archive_clean.columns, tweets_json_clean.columns

In [26]:
# Define
# Merge the twitter_enhanced_clean and tweets_json_clean together using 'tweet_id'
tweets_super_clean = tweets_json_clean.merge(twitter_archive_clean, how='inner', on='tweet_id')
# Test: let's see what columns we have now and if the merge is doing what we want it to do
#tweets_super_clean.head()
#tweets_super_clean.info()

In [27]:
# Define: before we move onto our second merge, we need to remove duplicate columns
# timestamp_x and timestamp_y show the same data. Timestamp_x is in the datetime format we want so we'll keep that one
# two columns for full text as well. We'll keep the first one
# Drop the columns
tweets_super_clean.drop(['timestamp_y', 'full_text_y'], axis=1, inplace=True)
# Rename the columns
tweets_super_clean.rename(columns={'timestamp_x':'timestamp', 'full_text_x': 'full_text'}, inplace=True)
# Test: verify our column remmoval was clean and successful
tweets_super_clean.columns

Index(['tweet_id', 'timestamp', 'favorite_count', 'retweet_count', 'full_text',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

### Second Merge

In [28]:
# Define
# We'll now merge tweets_super_clean with image_predictions_clean using tweet_ids
tweets_super_clean = tweets_super_clean.merge(image_predictions_clean, on='tweet_id', how='inner')
# Test: Confirm column names
tweets_super_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066 entries, 0 to 2065
Data columns (total 23 columns):
tweet_id              2066 non-null int64
timestamp             2066 non-null datetime64[ns]
favorite_count        2066 non-null int64
retweet_count         2066 non-null int64
full_text             2066 non-null object
rating_numerator      2066 non-null int64
rating_denominator    2066 non-null int64
name                  2066 non-null object
doggo                 2066 non-null object
floofer               2066 non-null object
pupper                2066 non-null object
puppo                 2066 non-null object
jpg_url               2066 non-null object
img_num               2066 non-null int64
p1                    2066 non-null object
p1_conf               2066 non-null float64
p1_dog                2066 non-null bool
p2                    2066 non-null object
p2_conf               2066 non-null float64
p2_dog                2066 non-null bool
p3                    2066 non-nu

### Create new category column 'stage' to replace 4 columns: doggo, pupper, floofer, puppo

In [29]:
# Define
# Create new stage column combining the 4 other dog stage columns and use 'mixed' when more than one stage is available
# combine all 4 columns
tweets_super_clean['stage'] = tweets_super_clean['doggo']+tweets_super_clean['floofer']+ \
                    tweets_super_clean['pupper']+tweets_super_clean['puppo']
# remove the None characters from all the strings
tweets_super_clean['stage'] = tweets_super_clean['stage'].str.replace('None', '')
# replace empty strings with NaN values
tweets_super_clean['stage'] = tweets_super_clean['stage'].replace('', np.nan)
# Fix values that include multiple stages and replace with 'mixed'
stages = {'doggopupper': 'mixed',
                'doggopuppo': 'mixed',
                'doggofloofer': 'mixed'}
def find_mixed(tweets_super_clean):
    if tweets_super_clean['stage'] in stages.keys():
        new_stage = stages[tweets_super_clean['stage']]
        return new_stage
    else:
        return tweets_super_clean['stage']    
tweets_super_clean['stage'] = tweets_super_clean.apply(find_mixed, axis=1)
# Test: confirm that all stage values are either: pupper, doggo, puppo, floofer, or mixed
tweets_super_clean['stage'].value_counts()

pupper     211
doggo       67
puppo       23
mixed       13
floofer      7
Name: stage, dtype: int64

### Drop old dog stage columns

In [30]:
# Define
# Drop 4 columns that we don't need anymore since all the information is in the 'stage' column
tweets_super_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)
# rearrange the new stage column before the image prediction columns
tweets_super_clean = tweets_super_clean[['tweet_id', 'timestamp', 'favorite_count', 'retweet_count', 'full_text',
       'rating_numerator', 'rating_denominator', 'name', 'stage', 'jpg_url', 'img_num',
       'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
       'p3_dog']]
# Test
tweets_super_clean.head()

,tweet_id,timestamp,favorite_count,retweet_count,full_text,rating_numerator,rating_denominator,name,stage,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,2017-08-01 16:23:56,37806,8244,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,NaN,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,2017-08-01 00:17:27,32488,6091,This is Tilly. She's just checking pup on you....,13,10,Tilly,NaN,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,2017-07-31 00:18:03,24452,4029,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,NaN,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,2017-07-30 15:58:51,41150,8389,This is Darla. She commenced a snooze mid meal...,13,10,Darla,NaN,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,2017-07-29 16:00:24,39341,9094,This is Franklin. He would like you to stop ca...,12,10,Franklin,NaN,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True


### Replace 'None' string values with proper NaN values for missing data

### Standardize denominator ratings to be out of 10

In [31]:
# Define
# Standardize all the denominators value to be at 10 for proper consistent rating scale
# Clean
tweets_super_clean.rating_denominator = 10
# Test
tweets_super_clean.rating_denominator.value_counts()

10    2066
Name: rating_denominator, dtype: int64

### Find incorrect names and replace with NaN values

In [32]:
# Define
# Correct all 'a', 'the', 'an', etc. dog names by replacing them with NaN values
# Bad names tend to start with lowercase so we'll put all the lower case names into a list of bad_names
bad_names = ['None']
# put all the names into a series
names_left = tweets_super_clean.name.value_counts()
for i in names_left.index:
    if i.islower():
        bad_names.append(i)
# iterate through our df and replace bad names with NaN values
for bad_name in bad_names:
    tweets_super_clean.name.replace(bad_name, np.nan, inplace=True)
tweets_super_clean.name.value_counts()

Oliver          10
Charlie         10
Lucy            10
Tucker          10
Penny           10
Cooper          10
Lola             8
Winston          8
Sadie            8
Bo               8
Toby             7
Daisy            7
Milo             6
Bailey           6
Stanley          6
Dave             6
Jax              6
Koda             6
Scout            6
Bella            6
Rusty            6
Louis            5
Chester          5
Leo              5
Buddy            5
Oscar            5
Larry            5
Alfie            5
Bruce            4
Cassie           4
                ..
Stormy           1
Stella           1
Shaggy           1
Al               1
Lugan            1
Willem           1
Traviss          1
Emma             1
Franq            1
Kramer           1
Howie            1
Rizzo            1
Goose            1
Edd              1
Ruffles          1
Mabel            1
Alexanderson     1
Stephan          1
Mitch            1
Vixen            1
Flash            1
Amber       

### Correct numerator ratings to account for edge cases such as '420', '1776', and other outliers

In [33]:
tweets_super_clean['new_rating'] = tweets_super_clean.full_text.str.extract('(1?[0-9]?.?([0-9]*)/10)', expand=True)
#tweets_super_clean['new_rating'] = tweets_super_clean['new_rating'].str[:-3]
tweets_super_clean.new_rating.value_counts()
# # (1?[0-9]?.?[0-9]+/{1}10)
# this row is a picture of snoop dog. Not a dog
# tweets_super_clean.query("new_rating == '420/10'")['jpg_url']
#tweets_super_clean.query("new_rating == '1776'")['jpg_url']

# earlier tweets more common to use 9/10 but every dog is 10/10 because every dog is a good boy/girl
#tweets_super_clean.query("new_rating == '9'")['timestamp'] 
#tweets_super_clean.query("new_rating == '1'")['full_text'].values
tweets_super_clean.query("new_rating > '15' and p1_dog == True")

#tweets_super_clean['new_rating'] = tweets_super_clean['new_rating'].astype(float)

tweets_super_clean[tweets_super_clean['rating_numerator'] > 15]['full_text'].values
tweets_super_clean.new_rating.value_counts()
#tweets_super_clean.info()

ValueError: Wrong number of items passed 2, placement implies 1

In [34]:
tweets_super_clean.rating_numerator.value_counts()

12      473
10      428
11      413
13      279
9       151
8        95
7        52
14       38
5        34
6        32
3        19
4        16
2         9
1         5
0         2
80        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
143       1
420       1
1776      1
15        1
27        1
45        1
75        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

## Quality Issues
#### *twitter_archive*
* ~~'None values are strings but should be NaN values. True for all three tables~~
* ~~Incorrect and missing names under 'name' column: 'None', 'a', 'the', 'an', etc.~~
* ~~Doggo, pupper, floofer, puppo cols will become redundant once we add new 'stage' column~~
* Rating numerators are under 10 but according to the twitter profile, all ratings should be above 10
* ~~There are rows where the 'rating_denominator' is lower or higher than 10. We need to standardize all rows to be out of 10.~~
* ~~Timestamp column is in string format instead of datetime.~~

#### *tweets_json*
* ~~'Id' column should be renamed to 'tweet_id' to be consistent with other two dataframes~~
* ~~duplicate 'created_at' column that is equivalent to twitter_archive 'timestamp' column. We'll keep one~~
* ~~we'll have duplicate 'full_text' columns as well after merging~~

#### *image_predictions*
* Dog breed predictions from the neural net in p1 give us results in inconsistent lower/upper case usage
* We get results from our neural net in p1 that don't include dogs such as 'desktop_computer', 'electric_fan', 'wild_boar'.
* Some prediction of dog breeds aren't actual dog breeds

## Tidiness Issues
* ~~twitter_archive has 2356 entries, tweets_json has 2340 entries, and image_predictions has 2075 entries. They all serve the same purpose of giving us more info about individual tweets. Should merge all three~~
* ~~In 'twitter_archive', the last four columns (doggo, floofer, pupper, puppo) are not always observed and best serve as a category. We should combine these 4 columns into one~~